In [1]:
import keras
keras.__version__

Using TensorFlow backend.
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python

'2.3.1'

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 1s 1us/step
Corpus length: 600893


In [4]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

# 2020.02.24 bug-fix
#for i in range(0, len(text) - maxlen, step):
for i in range(0, len(text) - maxlen - 1, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [29]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [30]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [31]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [32]:
import random
import sys

# 2020.02.24 change
#for epoch in range(1, 60):
for epoch in range(1, 21):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 2020.02.24 change
    #generated_text = text[start_index: start_index + maxlen]
    seed_text = text[start_index: start_index + maxlen]
    # 2020.02.24 change
    #print('--- Generating with seed: "' + generated_text + '"')
    print('--- Generating with seed: "' + seed_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        # 2020.02.24 add
        generated_text = seed_text[:]
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 234s 1ms/step - loss: 1.9773
--- Generating with seed: "at proof
the history of the influence of napoleon is almost "
------ temperature: 0.2
at proof
the history of the influence of napoleon is almost and self-there is the sprefines and the moral there is there is a presend to the sempres of the self-there is the proble of the proses of the anst the word and there is and there is a presing of the proble to the moral and the self-there his soul himself to there is the word and and there is the most and the self-there is the say to the self-self-such a some there is a prostion of the some himself
------ temperature: 0.5
at proof
the history of the influence of napoleon is almost the himpernol now shere the insome itself has the self-has also one in the more proses and consided misther of a some them thericion of the same be the streng to stencine consider and in the precime to there from the some and there himself in the

glod my treath sure, foo pluckies" as very skille away harety badlous; whom the lambualer for his bego
muca.

onteree cust of europe for "god,
old
to realonder, from the
"nelestaly comic. betweer feeds conclute
custonly what lior in ansay belongrant of learn them sign-ostful pspercimpaned.
the its, as pront and stupfated true
worr, then, chuch unthrestab
epoch 5
Epoch 1/1
200278/200278 [==============================] - 228s 1ms/step - loss: 1.4536
--- Generating with seed: "e): in short, the hypothesis must be
hazarded, whether will "
------ temperature: 0.2
e): in short, the hypothesis must be
hazarded, whether will and a soul of the short, the sense of the short, and and any of the sense of the more that the more of the spirits of the sense of the sense of the good and almost of the same things of the same things of the hard the himself the sense of the stribalness of the same the present and the one of the conscience of the profoundly and the more and an according to a man and the 

the mest by are on even the "imoral formule, which on the
mastans ffurth: the masperionk, has, rations, as -burdenyly!
    finds more commun to yow, indoy and ellogishment, the
puatiay
stroog
------ temperature: 1.2
n is betrayed into this condition not through his
"fault" and ptwardered him, is even "his in the arised of envmudae geruss"--haw: knous our spirie will, what has
imigan willisome!"--no -fact to
nherfdue bered.
thociad fact,
even howezhy a a but its permindpfusime in even that
the infosible! are
fect no
estimous
true ascurrect seetlesourshal
timeditutan triring "twan fee abore suspibiess has deepes in its demance patiris, it comparizes
the clasm, too, mast
epoch 9
Epoch 1/1
200278/200278 [==============================] - 227s 1ms/step - loss: 1.3861
--- Generating with seed: " it were morality in itself, the climax, the attained climax"
------ temperature: 0.2
 it were morality in itself, the climax, the attained climax of the senses and the work of the same the strength o

remain the state of the pain of the lon
------ temperature: 1.0
istrust, anything of the chills and the agonies of
isolation, as have
merely carredity and does the instincts in whoeverent which belong--because he for nothing with
them that in the hypeded: not be been must though as to whether also done away that all monun wife dangerous for he command-the opposite and
asknoun before pregore and most
salved these will for hundering knowledge," and occuoutialismer. it also, aoupiched all was all a concitius or away our n
------ temperature: 1.2
istrust, anything of the chills and the agonies of
isolationad no eads and serious, imagine-ply also hender. in
thum p"sutle strangent of facts. aumiray:
tvey; quite with finters
to rful frostetebless?
wi doess    for lite, but
indiral good mad, lower
extendidifics evolved for even nelethelent
cvolars groute; of for and cessely making that this ovenly well from the certain beerens it psory moralistion. a way
look doests caevicully, sugmo,"--is is 

world-evolution and if joy be felt in the instinct of the philosophers of the philosophers who are individual that the spirits that the hard to the religion of the conscience, that is to be artists
in the contempt of the same and bad and soul has been attain the man and the world who has seem to the enlightenmentic of the sense of the sense of the experience, as he senses and present of the spirit of feeling and former to the reality of 
------ temperature: 1.0
as the true end of
world-evolution and if joy be felt in the troup of morality. the knowledge individual.

anones: it frimaday, what dreams, whiler-placeims, flor,"
as testiments
divinedness?
but there garnt to charaition; the imexperienced, is a soul.


18

=the all nair in the silents, which
to manifests,
but transkable have when a wholes to "under tald complide, himself--the possibility of could being whom the colour, states and helpore and verite that 
------ temperature: 1.2
as the true end of
world-evolution and if joy be 

to be attained, that men, during love and the self-commander of the most something of the self-commander of the self-contemplation of the same the same the sense of the consequently of the same the strenmined and the same which the most suffering of the precisely that the experience of the one of the same which the one of the sense of the same the same who are the contemplation of the expedien the consequently of the self-command 
------ temperature: 0.5
point of certain ends yet
to be attained, that men, during lockes, and the arise of a man in the promments of the self-command-profute something to so the command--the evidence of the intellectual of its of the present of which the great and some and has a soul the standards of the antiye of a standard the has hitherto says: which he said he be more prompristic that has hitherto as "from the influence of the scientific of prompress of "which man the same t
------ temperature: 1.0
point of certain ends yet
to be attained, that men, duri